In [1]:
from topojson import Topology
import geopandas as gpd

gdf = gpd.read_file("https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_admin_0_countries.geojson")
africa = gdf.query('continent == "Africa"')

### Comparison simplification without Topology and with Topology

In [2]:
l = Topology(
    data=africa, 
    topology=False
).toposimplify(4).to_alt().properties(title='WITHOUT Topology')
r = Topology(
    data=africa, 
    topology=True
).toposimplify(4).to_alt().properties(title='WITH Topology')
l | r

alt.HConcatChart(...)

### Comparison no Topoquantization and with Topoquantization

In [3]:
l = Topology(africa).to_alt().properties(title='NO Topoquantization')
r = Topology(africa).topoquantize(100).to_alt().properties(title='WITH Topoquantization')
l | r

alt.HConcatChart(...)

### Comparison between Douglas-Peucker simplification and Visvalingam-Whyatt simplification

In [4]:
topo = Topology(africa) 

l = topo.toposimplify(
    epsilon=4, 
    simplify_with='shapely', 
    simplify_algorithm='dp'
).to_alt().properties(title=['Douglas-Peucker simplification'])


r = topo.toposimplify(
    epsilon=4, 
    simplify_with='simplification', 
    simplify_algorithm='vw'
).to_alt().properties(title='Visvalingam-Whyatt simplification')
l | r

alt.HConcatChart(...)

### Comparison datasize of raw GeoJSON and optimized TopoJSON

In [5]:
africa.to_file('africa.geo.json', driver='GeoJSON')

In [6]:
topo = Topology(
    data=africa, 
    simplify_with='simplification', 
    simplify_algorithm='vw', 
    toposimplify=4, 
    topoquantize=200
)
topo.to_json('africa.topo.json')

In [7]:
import os
size_geojson = os.stat('africa.geo.json').st_size
size_topojson = os.stat('africa.topo.json').st_size

print(f'africa.geo.json: {size_geojson/1000} KB')
print(f'africa.topo.json: {size_topojson/1000} KB')
print(f'factor smaller: {size_geojson/size_topojson} X')

africa.geo.json: 102.724 KB
africa.topo.json: 11.916 KB
factor smaller: 8.620678079892581 X


In [8]:
l = Topology(
    data=africa, 
    topology=False, 
    prequantize=False
).to_alt().properties(title=['Raw GeoJSON', '102.72 KB'])
r = topo.to_alt().properties(
    title=['Optimized TopoJSON', f'{size_topojson/1000} KB ({round(size_geojson/size_topojson, 2)}X smaller)']
)
l | r

alt.HConcatChart(...)

### Showcase multi-layer Topology

In [9]:
continent = africa.dissolve(by='continent', as_index=False)

a = Topology(
    data=[africa, continent],  
    object_name=['africa_countries', 'africa_continent']
)

l = a.toposimplify(1).to_alt(object_name='africa_countries').properties(title='Object `africa_countries`')
r = a.toposimplify(1).to_alt(object_name='africa_continent').properties(title='Object `africa_continent`')

import altair as alt
alt.data_transformers.consolidate_datasets = False
l | r

alt.HConcatChart(...)

### Showcase computing Topology using Jupyter Widgets

In [10]:
t = Topology(africa, prevent_oversimplify=False)
t.to_widget()

interactive(children=(FloatSlider(value=0.01, description='Toposimplify Factor', max=10.0, step=0.01, style=Sl…

<function topojson.utils.toposimpquant(epsilon, quant, algo, topo)>